In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

from sklearn.preprocessing import PolynomialFeatures, MinMaxScaler

from sklearn.model_selection import train_test_split

from sklearn.linear_model import LogisticRegression


In [16]:
#load Historic data csv into dataframe.
df = pd.read_csv('/Users/mercums/Documents/UMD/2021-2022/SEMESTER 2/INST 414/historic_data2_na.csv')
df


,aqi_class,aqi_class_recode,aqi,pm10,pm2.5,o3,timestamp_local,so2,no2,timestamp_utc,datetime,co,ts,city
0,ok,3,63,24.77,18.01,26.31,2022-03-01T00:00:00,3.59,53.84,2022-03-01T05:00:00,2022-03-01:05,606.03,1646110800,New York
1,ok,3,61,18.39,17.16,26.15,2022-02-28T23:00:00,2.04,51.71,2022-03-01T04:00:00,2022-03-01:04,556.06,1646107200,New York
2,ok,3,59,12.00,9.00,26.00,2022-02-28T22:00:00,0.50,49.58,2022-03-01T03:00:00,2022-03-01:03,506.09,1646103600,New York
3,ok,3,57,12.00,9.00,31.54,2022-02-28T21:00:00,0.50,44.79,2022-03-01T02:00:00,2022-03-01:02,556.47,1646100000,New York
4,good,4,34,11.00,8.20,46.00,2022-02-28T20:00:00,0.25,33.33,2022-03-01T01:00:00,2022-03-01:01,467.16,1646096400,New York
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12640,ok,3,67,29.00,20.00,14.50,2022-02-01T04:00:00,0.63,38.75,2022-02-01T09:00:00,2022-02-01:09,430.52,1643706000,Washington DC
12641,ok,3,67,24.88,19.67,10.33,2022-02-01T03:00:00,0.42,40.42,2022-02-01T08:00:00,2022-02-01:08,428.23,1643702400,Washington DC
12642,ok,3,66,20.76,19.33,6.17,2022-02-01T02:00:00,0.21,42.08,2022-02-01T07:00:00,2022-02-01:07,425.94,1643698800,Washington DC
12643,ok,3,65,16.64,12.48,2.00,2022-02-01T01:00:00,0.00,43.75,2022-02-01T06:00:00,2022-02-01:06,423.65,1643695200,Washington DC


In [23]:
#Assigning Data to X and Y
X = df[["pm10", "pm2.5", "o3", "so2", "no2", "co"]].values
y = df["aqi_class_recode"].values   #this recode limits the values the model will predict to just 4

print("Shape: ", X.shape, y.shape)

#create test train split from data
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)


Shape:  (12645, 6) (12645,)


Logistic Regression

In [29]:
regr = LogisticRegression(max_iter=4000, penalty='none', solver='lbfgs')

# Fit the model to the training data
regr.fit(X_train, y_train)

# score on train and test data, where we expect to see higher scores for training data
train_score = regr.score(X_train, y_train)
print("Training Data Score:", train_score)
test_score = regr.score(X_test, y_test)
print("Testing Data Score:", test_score)

Training Data Score: 0.8792576188969735
Testing Data Score: 0.8877292852624921


Feature Normalization

In [26]:
minmax = MinMaxScaler()
poly = PolynomialFeatures(degree=2)

minmax.fit(X_train)

X_train_scaled = minmax.transform(X_train)
X_train_poly = poly.fit_transform(X_train_scaled)


In [27]:
# Rescale the test data as well 
X_test_scaled = minmax.transform(X_test)
X_test_poly = poly.fit_transform(X_test_scaled)


Logistic Regression with New Features


In [30]:
regr = LogisticRegression(max_iter=5000, penalty='none', solver='lbfgs')

# Retrain using the min/max and polynomial normalized data
regr.fit(X_train_poly, y_train)


train_score = regr.score(X_train_poly, y_train)
test_score = regr.score(X_test_poly, y_test)

# score on train and test data, where we expect to see higher scores for training data
train_score = regr.score(X_train_poly, y_train)
print("Training Data Score:", train_score)
test_score = regr.score(X_test_poly, y_test)
print("Testing Data Score:", test_score)


Training Data Score: 0.8909627754929874
Testing Data Score: 0.8969006957621758


/Library/Frameworks/Python.framework/Versions/3.9/lib/python3.9/site-packages/sklearn/linear_model/_logistic.py:814: ConvergenceWarning: lbfgs failed to converge (status=1):
STOP: TOTAL NO. of ITERATIONS REACHED LIMIT.

Increase the number of iterations (max_iter) or scale the data as shown in:
    https://scikit-learn.org/stable/modules/preprocessing.html
Please also refer to the documentation for alternative solver options:
    https://scikit-learn.org/stable/modules/linear_model.html#logistic-regression
  n_iter_i = _check_optimize_result(
